In [210]:
import cv2 as cv
import numpy as np
import pandas as pd
import os
import shutil
import json
import random
import math
import matplotlib.pyplot as plt

In [283]:
DATASET_PATH = 'D:\\data_sets\\oral_photograph\\roboflow\\proyecto_odoto_v1.v6i.coco-segmentation\\valid'
TARGET_PATH = 'data_sets\\oral_tampering\\'

In [284]:
images = []
for root, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        if file.endswith('.jpg'):
            images.append(os.path.join(root, file))

In [285]:
with open(DATASET_PATH + '\\_annotations.coco.json', 'r', encoding='utf-8') as json_file:
    annotations = json.load(json_file)

In [286]:
categories_df = pd.DataFrame(annotations['categories'])
images_df = pd.DataFrame(annotations['images'])
annotations_df = pd.DataFrame(annotations['annotations'])

In [287]:
categories_df

,id,name,supercategory
0,0,enfermedades dentales,none
1,1,absceso,enfermedades dentales
2,2,caries cervical,enfermedades dentales
3,3,caries de dentina,enfermedades dentales
4,4,caries de esmalte,enfermedades dentales
5,5,caries interproximal,enfermedades dentales
6,6,caries profunda,enfermedades dentales
7,7,caries radicular,enfermedades dentales
8,8,caries rampante,enfermedades dentales
9,9,sarro,enfermedades dentales


In [288]:
images_df

,id,license,file_name,height,width,date_captured
0,0,1,031_00488_09_a091.rf.008611f0ddf57150c59641fc4...,640,640,2023-10-28T19:54:14+00:00
1,1,1,-25-_jpg.rf.006ad3ded66f894631fe64e99d629b28.jpg,640,640,2023-10-28T19:54:14+00:00
2,2,1,003_00043_08_a091.rf.0451ab3a31cf6430afe011b47...,640,640,2023-10-28T19:54:14+00:00
3,3,1,003_00035_10_a091.rf.07bff9e7e2d4f0e9193e95f09...,640,640,2023-10-28T19:54:14+00:00
4,4,1,19_jpg.rf.06252cb9dd77588918049d08d79bb6e7.jpg,640,640,2023-10-28T19:54:14+00:00
...,...,...,...,...,...,...
196,196,1,20230328_133818_jpg.rf.fb688ca77a9fea3153522df...,640,640,2023-10-28T19:54:14+00:00
197,197,1,005_00165_04_a091.rf.f5710782c6dd9166b962f1c2c...,640,640,2023-10-28T19:54:14+00:00
198,198,1,20230403_133755_jpg.rf.fe0484101abe878eba7f4ac...,640,640,2023-10-28T19:54:14+00:00
199,199,1,1019_tra_jpg.rf.ffd2ff25722ed9daa3c27e60c2417c...,640,640,2023-10-28T19:54:14+00:00


In [289]:
annotations_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,1,9,"[70, 299, 45.139, 164.49]",7424.911,"[[104.293, 409.205, 102.473, 424.788, 99.015, ...",0
1,1,1,9,"[112, 285, 48.233, 163.913]",7906.039,"[[127.408, 441.526, 130.867, 449.029, 133.779,...",0
2,2,1,9,"[155, 286, 49.689, 176.61]",8775.640,"[[158.897, 389.581, 162.173, 399.97, 163.811, ...",0
3,3,1,9,"[194, 291, 49.507, 177.765]",8800.642,"[[203.854, 447.874, 206.038, 460.572, 207.676,...",0
4,4,1,9,"[25, 364, 53.195, 177.495]",9441.876,"[[49.278, 517.261, 55.217, 522.87, 59.386, 530...",0
...,...,...,...,...,...,...,...
294,294,196,9,"[35, 370, 40, 160]",6400.000,"[[73.75, 425.625, 53.125, 370, 48.75, 371.875,...",0
295,295,196,9,"[56, 176, 27.5, 138.125]",3798.438,"[[69.375, 176.25, 63.75, 181.875, 61.25, 221.8...",0
296,296,199,3,"[222, 211, 64.249, 65.313]",4196.329,"[[232.432, 232.855, 221.724, 234.748, 226.763,...",0
297,297,199,3,"[177, 395, 23.936, 40.702]",974.254,"[[183.3, 435.42, 192.118, 425.955, 197.158, 41...",0


In [290]:
def create_point(size, sigma):
    mask = np.zeros((size[0], size[1]), dtype=np.float32)
    if sigma == 0:
        sigma = 1
    for i in range(size[0]):
        for j in range(size[1]):
            mask[i, j] = np.exp(-((i - size[0] // 2) ** 2 + (j - size[1] // 2) ** 2) / (2 * sigma ** 2))
    mask = 1 - mask
    mask = mask * 255
    return mask


def apply_opacity_mask(img, mask):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i, j, 3] = mask[i, j]
    return img



def join_alpha_channel(img):
    alpha_channel = img[:, :, 3]
    img = img[:, :, :3]
    b, g, r = cv.split(img)
    alpha_channel = alpha_channel / 255.0
    b = b * alpha_channel
    g = g * alpha_channel
    r = r * alpha_channel
    img = cv.merge((b, g, r))
    return img



def create_mask(shape, contours):
    mask = np.zeros(shape, dtype=np.uint8)
    cv.drawContours(mask, contours, -1, 255, -1)
    cv.drawContours(mask, contours, -1, 255, 2)
    return mask


def inpaint_diseases(img, mask):
    return cv.inpaint(img, mask, 3, cv.INPAINT_TELEA)

In [291]:
def remove_diseases(file_path, annotations):
    image = cv.imread(file_path)
    contours = []
    bboxes = []
    for annotation in annotations:
        bbox = np.array(annotation['bbox']).astype(np.int32)
        contour = np.array(annotation['segmentation'][0]).reshape(
            (-1, 1, 2)).astype(np.int32)
        contours.append(contour)
        bboxes.append(bbox)
    mask = create_mask(image.shape[:2], contours)
    return inpaint_diseases(image, mask)


def add_points(file_path, annotations):
    image = cv.imread(file_path)
    for annotation in annotations:
        bbox = np.array(annotation['bbox']).astype(np.int32)
        if bbox[2] < 2 or bbox[3] < 2:
            continue
        contour = np.array(annotation['segmentation'][0]).reshape(
            (-1, 1, 2)).astype(np.int32)
        result = image[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]
        roi = result.copy()
        contour = contour - bbox[:2]
        point = create_point(result.shape[:2], np.min(result.shape[:2]) // 4)
        mask = point.copy()
        roi = cv.cvtColor(roi, cv.COLOR_BGR2BGRA)
        roi = apply_opacity_mask(roi, mask)
        roi = join_alpha_channel(roi)
        image[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]] = roi

    image = cv.medianBlur(image, 5)
    return image

In [292]:
# if os.path.exists(TARGET_PATH):
#     shutil.rmtree(TARGET_PATH)

# os.mkdir(TARGET_PATH)

In [293]:
# file_name = '1202_tra_jpg.rf.96333138f9c2bad11976a1ad09d99456.jpg'
# file_path = DATASET_PATH + '\\' + file_name
# image_dict = images_df[images_df['file_name'] == file_name].to_dict('records')[0]
# annotations = annotations_df[annotations_df['image_id'] == image_dict['id']].to_dict('records')
# result = remove_diseases(DATASET_PATH + '\\' + file_name, annotations)
# cv.imwrite('inpaint_' + file_name, result)
# result = add_points(TARGET_PATH + '\\' + 'inpaint_' +
#                     file_name, annotations)
# cv.imwrite('inpaint_point_' + file_name, result)

In [294]:
for file_name in os.listdir(DATASET_PATH):
    if file_name.endswith('.jpg'):
        if os.path.exists(TARGET_PATH + 'inpaint_' + file_name):
            os.remove(TARGET_PATH + 'inpaint_' + file_name)
        if os.path.exists(TARGET_PATH + 'inpaint_point_' + file_name):
            os.remove(TARGET_PATH + 'inpaint_point_' + file_name)
        image_dict = images_df[images_df['file_name'] == file_name].to_dict('records')[0]
        annotations = annotations_df[annotations_df['image_id'] == image_dict['id']].to_dict('records')
        if len(annotations) == 0:
            continue
        result = remove_diseases(DATASET_PATH + '\\' + file_name, annotations)
        cv.imwrite(TARGET_PATH + 'inpaint_' + file_name, result)
        result = add_points(TARGET_PATH + '\\' + 'inpaint_' +
                            file_name, annotations)
        cv.imwrite(TARGET_PATH + 'inpaint_point_' + file_name, result)